# Monotonicity Pattern Analysis

**Goal**: Find patterns in (k, d) values that guarantee diameter monotonicity for each coset type.

We want to extract rules like:
- "For coset X, diameter is always monotonic regardless of k, d"
- "For coset Y, monotonicity holds when k < threshold"
- "For coset Z, monotonicity fails only when k + d > n/2"

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Load all CSVs from results_perm1
results_dir = Path("results_perm1")
dfs = []

for group_dir in results_dir.iterdir():
    if group_dir.is_dir():
        csv_path = group_dir / "data.csv"
        if csv_path.exists():
            df = pd.read_csv(csv_path)
            df['group'] = group_dir.name
            dfs.append(df)
            print(f"Loaded {len(df)} rows from {group_dir.name}")

df = pd.concat(dfs, ignore_index=True)
print(f"\nTotal: {len(df)} rows")
print(f"Cosets: {sorted(df['coset'].unique())}")
print(f"k range: {df['k'].min()} to {df['k'].max()}")
print(f"d range: {df['d'].min()} to {df['d'].max()}")

Loaded 2298 rows from different
Loaded 281 rows from full_graph
Loaded 2138 rows from then
Loaded 2144 rows from repeats

Total: 6861 rows
Cosets: ['011Repeats', '012Repeats', '0then1then2', '0then1then2then3', '0then1then2then3then4', '2Different', '3Different', '4Different', 'Binary01Repeats', 'Binary01Repeats_1', 'Binary0then1', 'FullGraph']
k range: 0 to 18
d range: 1 to 10


In [2]:
def check_monotonicity(group_df):
    """Check if diameter is monotonically increasing as n increases."""
    sorted_df = group_df.sort_values('n')
    diameters = sorted_df['diameter'].values
    n_values = sorted_df['n'].values
    
    violations = []
    for i in range(1, len(diameters)):
        if diameters[i] < diameters[i-1]:
            violations.append({
                'n_prev': int(n_values[i-1]),
                'n_curr': int(n_values[i]),
                'diameter_prev': int(diameters[i-1]),
                'diameter_curr': int(diameters[i]),
            })
    
    return {
        'is_monotonic': len(violations) == 0,
        'num_violations': len(violations),
        'violations': violations,
        'n_min': int(n_values.min()),
        'n_max': int(n_values.max()),
    }

# Analyze all (coset, k, d) combinations
results = []
for (coset, k, d), group_df in df.groupby(['coset', 'k', 'd']):
    analysis = check_monotonicity(group_df)
    results.append({
        'coset': coset,
        'k': k,
        'd': d,
        'group': group_df['group'].iloc[0],
        'is_monotonic': analysis['is_monotonic'],
        'num_violations': analysis['num_violations'],
        'n_min': analysis['n_min'],
        'n_max': analysis['n_max'],
        'num_points': len(group_df),
    })

results_df = pd.DataFrame(results)
print(f"Analyzed {len(results_df)} unique (coset, k, d) combinations")

Analyzed 1080 unique (coset, k, d) combinations


## 1. Overall Summary by Coset

In [3]:
# Summary by coset
coset_summary = results_df.groupby('coset').agg({
    'is_monotonic': ['sum', 'count'],
    'num_violations': 'sum'
}).reset_index()
coset_summary.columns = ['coset', 'monotonic_count', 'total_count', 'total_violations']
coset_summary['non_monotonic_count'] = coset_summary['total_count'] - coset_summary['monotonic_count']
coset_summary['monotonic_pct'] = (coset_summary['monotonic_count'] / coset_summary['total_count'] * 100).round(1)
coset_summary = coset_summary.sort_values('monotonic_pct', ascending=False)

print("=" * 70)
print("MONOTONICITY SUMMARY BY COSET")
print("=" * 70)
display(coset_summary[['coset', 'monotonic_count', 'non_monotonic_count', 'total_count', 'monotonic_pct']])

MONOTONICITY SUMMARY BY COSET


,coset,monotonic_count,non_monotonic_count,total_count,monotonic_pct
5,2Different,85,0,85,100.0
11,FullGraph,26,39,65,40.0
7,4Different,31,54,85,36.5
2,0then1then2,30,65,95,31.6
6,3Different,26,59,85,30.6
4,0then1then2then3then4,29,66,95,30.5
3,0then1then2then3,29,66,95,30.5
8,Binary01Repeats,29,66,95,30.5
9,Binary01Repeats_1,29,66,95,30.5
10,Binary0then1,29,66,95,30.5


In [4]:
# Identify cosets that are ALWAYS monotonic
always_monotonic = coset_summary[coset_summary['monotonic_pct'] == 100]['coset'].tolist()
never_monotonic = coset_summary[coset_summary['monotonic_pct'] == 0]['coset'].tolist()
mixed = coset_summary[(coset_summary['monotonic_pct'] > 0) & (coset_summary['monotonic_pct'] < 100)]['coset'].tolist()

print(f"\n{'='*70}")
print("COSET CLASSIFICATION")
print(f"{'='*70}")
print(f"\nALWAYS MONOTONIC ({len(always_monotonic)} cosets):")
for c in always_monotonic:
    print(f"  - {c}")

print(f"\nNEVER MONOTONIC ({len(never_monotonic)} cosets):")
for c in never_monotonic:
    print(f"  - {c}")

print(f"\nMIXED - depends on (k, d) ({len(mixed)} cosets):")
for c in mixed:
    pct = coset_summary[coset_summary['coset'] == c]['monotonic_pct'].values[0]
    print(f"  - {c} ({pct}% monotonic)")


COSET CLASSIFICATION

ALWAYS MONOTONIC (1 cosets):
  - 2Different

NEVER MONOTONIC (0 cosets):

MIXED - depends on (k, d) (11 cosets):
  - FullGraph (40.0% monotonic)
  - 4Different (36.5% monotonic)
  - 0then1then2 (31.6% monotonic)
  - 3Different (30.6% monotonic)
  - 0then1then2then3then4 (30.5% monotonic)
  - 0then1then2then3 (30.5% monotonic)
  - Binary01Repeats (30.5% monotonic)
  - Binary01Repeats_1 (30.5% monotonic)
  - Binary0then1 (30.5% monotonic)
  - 012Repeats (29.5% monotonic)
  - 011Repeats (28.4% monotonic)


## 2. Heatmaps: Monotonicity by (k, d) for Each Coset

In [7]:
# Create heatmaps for cosets with mixed results
mixed_cosets = coset_summary[(coset_summary['monotonic_pct'] > 0) & 
                              (coset_summary['monotonic_pct'] < 100)]['coset'].tolist()

if len(mixed_cosets) == 0:
    print("No mixed cosets - all cosets are either always or never monotonic!")
else:
    for coset_name in mixed_cosets:
        coset_data = results_df[results_df['coset'] == coset_name].copy()
        
        # Convert boolean to int explicitly (True=1, False=0)
        coset_data['mono_int'] = coset_data['is_monotonic'].astype(int)
        
        # Pivot to create k x d matrix
        pivot = coset_data.pivot_table(
            index='k', 
            columns='d', 
            values='mono_int', 
            aggfunc='first'
        ).fillna(-1)  # -1 for missing combinations
        
        # Create heatmap
        fig = go.Figure(data=go.Heatmap(
            z=pivot.values,
            x=[f'd={d}' for d in pivot.columns],
            y=[f'k={k}' for k in pivot.index],
            colorscale=[
                [0, 'lightgray'],      # -1 (N/A) 
                [0.5, 'red'],          # 0 (non-monotonic)
                [1, 'green']           # 1 (monotonic)
            ],
            zmin=-1, zmax=1,
            hovertemplate='k=%{y}<br>d=%{x}<br>monotonic=%{z}<extra></extra>',
            colorbar=dict(tickvals=[-1, 0, 1], ticktext=['N/A', 'No', 'Yes'])
        ))
        
        mono_count = (coset_data['is_monotonic'] == True).sum()
        total_count = len(coset_data)
        
        fig.update_layout(
            title=f'{coset_name}: Monotonicity by (k, d) - {mono_count}/{total_count} monotonic',
            xaxis_title='d',
            yaxis_title='k',
            height=500
        )
        fig.show()

## 3. Pattern Extraction: Rules for Monotonicity

In [8]:
def extract_patterns(coset_name, coset_data):
    """Try to extract rules that predict monotonicity for a coset."""
    patterns = []
    
    mono = coset_data[coset_data['is_monotonic'] == True]
    non_mono = coset_data[coset_data['is_monotonic'] == False]
    
    if len(non_mono) == 0:
        patterns.append("ALWAYS MONOTONIC: All (k, d) combinations are monotonic")
        return patterns
    
    if len(mono) == 0:
        patterns.append("NEVER MONOTONIC: No (k, d) combinations are monotonic")
        return patterns
    
    # Check if k value determines monotonicity
    k_mono = mono['k'].unique()
    k_non_mono = non_mono['k'].unique()
    k_only_mono = set(k_mono) - set(k_non_mono)
    k_only_non_mono = set(k_non_mono) - set(k_mono)
    
    if len(k_only_mono) > 0:
        patterns.append(f"k in {sorted(k_only_mono)} → ALWAYS monotonic")
    if len(k_only_non_mono) > 0:
        patterns.append(f"k in {sorted(k_only_non_mono)} → NEVER monotonic")
    
    # Check if d value determines monotonicity
    d_mono = mono['d'].unique()
    d_non_mono = non_mono['d'].unique()
    d_only_mono = set(d_mono) - set(d_non_mono)
    d_only_non_mono = set(d_non_mono) - set(d_mono)
    
    if len(d_only_mono) > 0:
        patterns.append(f"d in {sorted(d_only_mono)} → ALWAYS monotonic")
    if len(d_only_non_mono) > 0:
        patterns.append(f"d in {sorted(d_only_non_mono)} → NEVER monotonic")
    
    # Check threshold patterns
    if len(mono) > 0 and len(non_mono) > 0:
        # k threshold
        if mono['k'].max() < non_mono['k'].min():
            patterns.append(f"k < {non_mono['k'].min()} → monotonic")
        elif mono['k'].min() > non_mono['k'].max():
            patterns.append(f"k > {non_mono['k'].max()} → monotonic")
        
        # d threshold
        if mono['d'].max() < non_mono['d'].min():
            patterns.append(f"d < {non_mono['d'].min()} → monotonic")
        elif mono['d'].min() > non_mono['d'].max():
            patterns.append(f"d > {non_mono['d'].max()} → monotonic")
    
    # Check k + d pattern
    mono_sum = mono['k'] + mono['d']
    non_mono_sum = non_mono['k'] + non_mono['d']
    if mono_sum.max() < non_mono_sum.min():
        patterns.append(f"k + d < {non_mono_sum.min()} → monotonic")
    elif mono_sum.min() > non_mono_sum.max():
        patterns.append(f"k + d > {non_mono_sum.max()} → monotonic")
    
    # Check k * d pattern
    mono_prod = mono['k'] * mono['d']
    non_mono_prod = non_mono['k'] * non_mono['d']
    if mono_prod.max() < non_mono_prod.min():
        patterns.append(f"k * d < {non_mono_prod.min()} → monotonic")
    
    # Check |k - d| pattern
    mono_diff = abs(mono['k'] - mono['d'])
    non_mono_diff = abs(non_mono['k'] - non_mono['d'])
    if mono_diff.max() < non_mono_diff.min():
        patterns.append(f"|k - d| < {non_mono_diff.min()} → monotonic")
    
    if len(patterns) == 0:
        patterns.append("No simple pattern found - complex interaction between k and d")
    
    return patterns

# Extract patterns for each coset
print("=" * 70)
print("EXTRACTED PATTERNS FOR MONOTONICITY")
print("=" * 70)

all_patterns = {}
for coset_name in sorted(results_df['coset'].unique()):
    coset_data = results_df[results_df['coset'] == coset_name]
    patterns = extract_patterns(coset_name, coset_data)
    all_patterns[coset_name] = patterns
    
    print(f"\n{coset_name}:")
    for p in patterns:
        print(f"  • {p}")

EXTRACTED PATTERNS FOR MONOTONICITY

011Repeats:
  • k in [np.int64(12), np.int64(13)] → ALWAYS monotonic
  • d in [np.int64(2)] → ALWAYS monotonic

012Repeats:
  • k in [np.int64(12), np.int64(13)] → ALWAYS monotonic
  • d in [np.int64(2)] → ALWAYS monotonic

0then1then2:
  • k in [np.int64(10), np.int64(12), np.int64(13)] → ALWAYS monotonic
  • d in [np.int64(2)] → ALWAYS monotonic

0then1then2then3:
  • k in [np.int64(12), np.int64(13)] → ALWAYS monotonic
  • d in [np.int64(2)] → ALWAYS monotonic

0then1then2then3then4:
  • k in [np.int64(12), np.int64(13)] → ALWAYS monotonic
  • d in [np.int64(2)] → ALWAYS monotonic

2Different:
  • ALWAYS MONOTONIC: All (k, d) combinations are monotonic

3Different:
  • k in [np.int64(16), np.int64(18)] → ALWAYS monotonic
  • d in [np.int64(2)] → ALWAYS monotonic

4Different:
  • k in [np.int64(16), np.int64(18)] → ALWAYS monotonic
  • d in [np.int64(2)] → ALWAYS monotonic

Binary01Repeats:
  • k in [np.int64(12), np.int64(13)] → ALWAYS monotonic


## 4. Detailed Analysis of Non-Monotonic Cases

In [9]:
# For non-monotonic cases, look at WHERE the violations occur (at what n values)
non_mono_detailed = []

for (coset, k, d), group_df in df.groupby(['coset', 'k', 'd']):
    analysis = check_monotonicity(group_df)
    if not analysis['is_monotonic']:
        for v in analysis['violations']:
            non_mono_detailed.append({
                'coset': coset,
                'k': k,
                'd': d,
                'n_prev': v['n_prev'],
                'n_curr': v['n_curr'],
                'diameter_prev': v['diameter_prev'],
                'diameter_curr': v['diameter_curr'],
                'decrease': v['diameter_prev'] - v['diameter_curr']
            })

if len(non_mono_detailed) > 0:
    violations_df = pd.DataFrame(non_mono_detailed)
    
    print("=" * 70)
    print("WHERE DO VIOLATIONS OCCUR?")
    print("=" * 70)
    
    # At what n values do violations tend to happen?
    print("\nViolations by n_curr (where diameter decreases):")
    n_counts = violations_df['n_curr'].value_counts().sort_index()
    print(n_counts.to_string())
    
    print(f"\nMost common n where violations occur: n={violations_df['n_curr'].mode().values[0]}")
    print(f"Average decrease magnitude: {violations_df['decrease'].mean():.2f}")
    print(f"Max decrease: {violations_df['decrease'].max()}")
else:
    print("No violations found!")

WHERE DO VIOLATIONS OCCUR?

Violations by n_curr (where diameter decreases):
n_curr
6      48
8      86
9      54
10    163
11      3
12    371
14    377
15    317
16     43
17      3
18     87
19      3
20     93

Most common n where violations occur: n=14
Average decrease magnitude: 16.63
Max decrease: 106


In [10]:
# Violations breakdown by coset
if len(non_mono_detailed) > 0:
    print("\nViolations by coset:")
    coset_violations = violations_df.groupby('coset').agg({
        'n_curr': 'count',
        'decrease': ['mean', 'max']
    }).round(2)
    coset_violations.columns = ['num_violations', 'avg_decrease', 'max_decrease']
    coset_violations = coset_violations.sort_values('num_violations', ascending=False)
    display(coset_violations)


Violations by coset:


,num_violations,avg_decrease,max_decrease
coset,,,
3Different,221,14.14,73
4Different,206,25.82,106
012Repeats,149,18.31,58
Binary0then1,146,10.32,37
Binary01Repeats_1,145,10.93,37
Binary01Repeats,143,11.20,38
0then1then2,143,16.41,53
0then1then2then3,143,20.09,64
011Repeats,140,11.94,43


## 5. Statistical Analysis: k and d Distributions

In [11]:
# Compare k and d distributions between monotonic and non-monotonic cases
mono_cases = results_df[results_df['is_monotonic'] == True]
non_mono_cases = results_df[results_df['is_monotonic'] == False]

print("=" * 70)
print("STATISTICAL COMPARISON: MONOTONIC vs NON-MONOTONIC")
print("=" * 70)

if len(non_mono_cases) > 0:
    print(f"\nMonotonic cases: {len(mono_cases)}")
    print(f"  k: mean={mono_cases['k'].mean():.2f}, median={mono_cases['k'].median()}, range=[{mono_cases['k'].min()}, {mono_cases['k'].max()}]")
    print(f"  d: mean={mono_cases['d'].mean():.2f}, median={mono_cases['d'].median()}, range=[{mono_cases['d'].min()}, {mono_cases['d'].max()}]")
    
    print(f"\nNon-monotonic cases: {len(non_mono_cases)}")
    print(f"  k: mean={non_mono_cases['k'].mean():.2f}, median={non_mono_cases['k'].median()}, range=[{non_mono_cases['k'].min()}, {non_mono_cases['k'].max()}]")
    print(f"  d: mean={non_mono_cases['d'].mean():.2f}, median={non_mono_cases['d'].median()}, range=[{non_mono_cases['d'].min()}, {non_mono_cases['d'].max()}]")
else:
    print("\nAll cases are monotonic!")

STATISTICAL COMPARISON: MONOTONIC vs NON-MONOTONIC

Monotonic cases: 398
  k: mean=7.45, median=8.0, range=[0, 18]
  d: mean=3.27, median=2.0, range=[1, 10]

Non-monotonic cases: 682
  k: mean=4.21, median=4.0, range=[0, 17]
  d: mean=4.73, median=4.0, range=[1, 10]


In [12]:
# Visualize k distribution for monotonic vs non-monotonic
if len(non_mono_cases) > 0:
    fig = make_subplots(rows=1, cols=2, subplot_titles=['k distribution', 'd distribution'])
    
    # k distribution
    fig.add_trace(go.Histogram(x=mono_cases['k'], name='Monotonic', opacity=0.7, marker_color='green'), row=1, col=1)
    fig.add_trace(go.Histogram(x=non_mono_cases['k'], name='Non-monotonic', opacity=0.7, marker_color='red'), row=1, col=1)
    
    # d distribution
    fig.add_trace(go.Histogram(x=mono_cases['d'], name='Monotonic', opacity=0.7, marker_color='green', showlegend=False), row=1, col=2)
    fig.add_trace(go.Histogram(x=non_mono_cases['d'], name='Non-monotonic', opacity=0.7, marker_color='red', showlegend=False), row=1, col=2)
    
    fig.update_layout(barmode='overlay', title='Distribution of k and d: Monotonic vs Non-monotonic', height=400)
    fig.show()

## 6. Summary: Guaranteed Monotonicity Rules

In [13]:
print("=" * 70)
print("FINAL SUMMARY: RULES FOR GUARANTEED MONOTONICITY")
print("=" * 70)

print("\n" + "="*70)
print("GROUP 1: ALWAYS MONOTONIC (no restrictions needed)")
print("="*70)
for coset in always_monotonic:
    count = len(results_df[results_df['coset'] == coset])
    print(f"  ✓ {coset} ({count} combinations tested)")

print("\n" + "="*70)
print("GROUP 2: CONDITIONALLY MONOTONIC (rules apply)")
print("="*70)
for coset in mixed:
    print(f"\n  {coset}:")
    for p in all_patterns[coset]:
        print(f"    • {p}")

print("\n" + "="*70)
print("GROUP 3: NEVER MONOTONIC (avoid these)")
print("="*70)
for coset in never_monotonic:
    count = len(results_df[results_df['coset'] == coset])
    print(f"  ✗ {coset} ({count} combinations tested)")

if len(never_monotonic) == 0:
    print("  (none)")

FINAL SUMMARY: RULES FOR GUARANTEED MONOTONICITY

GROUP 1: ALWAYS MONOTONIC (no restrictions needed)
  ✓ 2Different (85 combinations tested)

GROUP 2: CONDITIONALLY MONOTONIC (rules apply)

  FullGraph:
    • k in [np.int64(10)] → ALWAYS monotonic
    • d in [np.int64(2), np.int64(10)] → ALWAYS monotonic

  4Different:
    • k in [np.int64(16), np.int64(18)] → ALWAYS monotonic
    • d in [np.int64(2)] → ALWAYS monotonic

  0then1then2:
    • k in [np.int64(10), np.int64(12), np.int64(13)] → ALWAYS monotonic
    • d in [np.int64(2)] → ALWAYS monotonic

  3Different:
    • k in [np.int64(16), np.int64(18)] → ALWAYS monotonic
    • d in [np.int64(2)] → ALWAYS monotonic

  0then1then2then3then4:
    • k in [np.int64(12), np.int64(13)] → ALWAYS monotonic
    • d in [np.int64(2)] → ALWAYS monotonic

  0then1then2then3:
    • k in [np.int64(12), np.int64(13)] → ALWAYS monotonic
    • d in [np.int64(2)] → ALWAYS monotonic

  Binary01Repeats:
    • k in [np.int64(12), np.int64(13)] → ALWAYS mon

In [ ]:
# Export patterns to a summary file
summary_lines = ["# Monotonicity Rules for perm_type=1\n"]
summary_lines.append("\n## Always Monotonic\n")
for coset in always_monotonic:
    summary_lines.append(f"- {coset}\n")

summary_lines.append("\n## Conditionally Monotonic\n")
for coset in mixed:
    summary_lines.append(f"\n### {coset}\n")
    for p in all_patterns[coset]:
        summary_lines.append(f"- {p}\n")

summary_lines.append("\n## Never Monotonic\n")
for coset in never_monotonic:
    summary_lines.append(f"- {coset}\n")

with open('monotonicity_rules.md', 'w') as f:
    f.writelines(summary_lines)

print("Rules exported to monotonicity_rules.md")